In [1]:
import networkx as nx
import numpy as np
from scipy.integrate import odeint, solve_ivp
from copy import copy, deepcopy
import sys
np.set_printoptions(threshold=sys.maxsize)
import matplotlib.pyplot as plt
import time as time
from Classes.Analysis import Analysis

In [2]:
def read_graph(filename):
    return nx.read_gexf(filename)

def filter_node_type(graph, node_type):
    return [n for n in graph.nodes if graph.nodes[n]['node_type'] == node_type]

def turn_string_to_dictionary(string):
    dictionary = {}
    for row in string.split('\n'):
        row = row.split(':')
        if len(row) == 2:
            dictionary[row[0]] = row[1]
    return dictionary

graph = read_graph('example.gexf')
phage_nodes = filter_node_type(graph, 'P')
bacteria_nodes = filter_node_type(graph, 'B')
resource_nodes = filter_node_type(graph, 'R')
environemnt_nodes = filter_node_type(graph, 'E')
environemnt_data = turn_string_to_dictionary(graph.nodes[environemnt_nodes[0]]['data'])
simulation_time = round(float(environemnt_data['Simulation Time']))
time_step = float(environemnt_data['Time Step'])
M = int(environemnt_data['M'])
max_length = max(len(phage_nodes), len(bacteria_nodes), len(resource_nodes))

N_initial = np.zeros(max_length)
U_initial = np.zeros(max_length)
I_initial = np.zeros((max_length, M))
P_initial = np.zeros(max_length)

for i, node in enumerate(phage_nodes):
    data = turn_string_to_dictionary(graph.nodes[node]['data'])
    P_initial[i] = float(data['Initial Population'])
for i, node in enumerate(bacteria_nodes):
    data = turn_string_to_dictionary(graph.nodes[node]['data'])
    U_initial[i] = float(data['Initial Population'])
for i, node in enumerate(resource_nodes):
    data = turn_string_to_dictionary(graph.nodes[node]['data'])
    N_initial[i] = float(data['Initial Concentration'])

e_matrix = np.zeros((max_length, max_length))
v_matrix = np.zeros((max_length, max_length))
r_matrix = np.zeros((max_length, max_length))
K_matrix = np.zeros((max_length, max_length))
M_matrix = np.zeros((max_length, max_length))
tau_matrix = np.zeros((max_length, max_length))
B_matrix = np.zeros((max_length, max_length))
for u, v, d in graph.edges(data=True):
    if u in resource_nodes and v in bacteria_nodes or u in bacteria_nodes and v in resource_nodes:
        if(u in resource_nodes):
            u_index = resource_nodes.index(u)
            v_index = bacteria_nodes.index(v)
        else:
            u_index = resource_nodes.index(v)
            v_index = bacteria_nodes.index(u)
        e_matrix[u_index, v_index] = float(turn_string_to_dictionary(d['data'])['e'])
        v_matrix[u_index, v_index] = float(turn_string_to_dictionary(d['data'])['v'])
        K_matrix[u_index, v_index] = float(turn_string_to_dictionary(d['data'])['K'])
    elif u in phage_nodes and v in bacteria_nodes or u in bacteria_nodes and v in phage_nodes:
        if(u in phage_nodes):
            u_index = phage_nodes.index(u)
            v_index = bacteria_nodes.index(v)
        else:
            u_index = phage_nodes.index(v)
            v_index = bacteria_nodes.index(u)
        r_matrix[u_index, v_index] = float(turn_string_to_dictionary(d['data'])['r'])
        tau_matrix[u_index, v_index] = float(turn_string_to_dictionary(d['data'])['tau'])
        B_matrix[u_index, v_index] = float(turn_string_to_dictionary(d['data'])['Burst Size'])

simulation_time = 2
time_step = 0.1
t = np.linspace(0, round(simulation_time), abs(round(simulation_time/time_step) + 1))
y0_flattened = np.concatenate([N_initial.flatten(), U_initial.flatten(), I_initial.flatten(), P_initial.flatten()])

Graph with 10 nodes and 9 edges
['B_node_names', 'E_node_names', 'M', 'P_node_names', 'R_node_names', 'Simulation Time', 'Temperature', 'Time Step', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', 'add_environment_data', 'get_initial_pop_conc_of_nodes', 'get_nodes_of_type', 'graph', 'graph_location', 'load_graph', 'max_of_node_type', 'pH', 'turn_string_to_dictionary']


In [5]:
def new_system2(t, Y, M, e_matrix, v_matrix, K_matrix, r_matrix, tau_matrix, B_matrix, dimension):
    N = Y[:dimension]
    U = Y[dimension:dimension*2]
    I = Y[dimension*2:dimension*2 + dimension*M].reshape(dimension, M)
    P = Y[dimension*2 + dimension*M:]
    new_N = (e_matrix @ g(N, v_matrix, K_matrix)) @ (U + np.sum(I, axis=1))
    new_U = (g(N, v_matrix, K_matrix) @ U) - ((r_matrix @ U) * P)
    M_tau = np.nan_to_num(M / tau_matrix, nan=0, posinf=0, neginf=0)
    new_I = np.zeros_like(I)
    for column in range(M):
        if column == 0:
            new_I[:, column] = ((r_matrix @ U) @ P) - (M_tau @ I[:, column])
        else:
            new_I[:, column] = M_tau[:, column] @ (I[:, column] - I[:, column-1])
    new_P = ((B_matrix @ M_tau) @ I[:, -1]) - ((r_matrix @ (U + np.sum(I, axis=1))) @ P)
    y0_flattened = np.concatenate([new_N.flatten(), new_U.flatten(), new_I.flatten(), new_P.flatten()])

    return y0_flattened

def g(N, v, K):
    return (N @ v) / (N + K)

time3 = time.time()
solved = solve_ivp(new_system2, (0, simulation_time), y0_flattened, args=(M, e_matrix, v_matrix, K_matrix, r_matrix, tau_matrix, B_matrix, max_length), t_eval=t)
time4 = time.time()
total_time_taken = time4 - time3
solved_y = solved.y
N = solved_y[:max_length]
U = solved_y[max_length:max_length*2]
I = solved_y[max_length*2:max_length*2 + max_length*M]
P = solved_y[max_length*2 + max_length*M:]
print(U)

[[11.         13.70702514 75.35081498]
 [12.          9.29298643  4.1260307 ]
 [13.         12.82138335 12.52158846]
 [ 0.         -0.47012081 -1.21561703]]


/var/folders/gg/_vl162cs4zs99kls4stdr7xc0000gn/T/ipykernel_4313/119910788.py:8: RuntimeWarning: divide by zero encountered in divide
  M_tau = np.nan_to_num(M / tau_matrix, nan=0, posinf=0, neginf=0)
